# Interpretacja modelu

In [1]:
# Import wymaganych bibliotek
from  sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
os.chdir('../')

In [3]:
# puść ten kod, 
# jeżeli wywołujesz plik  w folderze rozwiąznaia, 
# a ramka danych znajduje się w folderze data
import os 
os.chdir('../')

In [4]:
# Pobranie danych

df = pd.read_csv('data/Cancer_Data.csv')
train_x = pd.read_csv('data/cancer_train_x.csv').iloc[:,1:]
test_x = pd.read_csv('data/cancer_test_x.csv').iloc[:,1:]
train_y = pd.read_csv('data/cancer_train_y.csv')['target']
test_y = pd.read_csv('data/cancer_test_y.csv')['target']

## Estymacja modelu
Regularyzacja jest metodą ograniczającą przeuczenie. Nakładana jest kara na funkcję celu, która minimalizuje nadmierne dopasowanie. 
W modelu regresji logistycznej mamy regularyzację wg parametrów l1 oraz l2. Siła regularyzacji ustalana jest parametrem C, gdzie niższe wartości, dają silniejszą regularyzację. 

Więcej informacji można znaleźć tu: https://medium.com/@rithpansanga/logistic-regression-and-regularization-avoiding-overfitting-and-improving-generalization-e9afdcddd09d

In [5]:
## dodanie zmiennej kategorycznej
train_x['perimeter_se_kat'] = (train_x['perimeter_se'] >=df['perimeter_se'].mean()).astype(int)
test_x['perimeter_se_kat'] = (test_x['perimeter_se'] >=df['perimeter_se'].mean()).astype(int)

In [6]:
# Usunięcie wersji nominalnej zmiennej
del train_x['perimeter_se']
del test_x['perimeter_se']

In [8]:
# Stworzenie obiektu modelu
model_1 = LogisticRegression(C=0.4)

In [9]:
# Estymacja 
model_1.fit(train_x, train_y)

LogisticRegression(C=0.4)

In [10]:
#Współczynniki modelu
model_1.coef_

array([[ 0.18556677,  0.67126629,  0.01305189,  0.22397476, -0.01895326]])

In [11]:
# Nazwy zmiennych
model_1.feature_names_in_

array(['perimeter_worst', 'compactness_worst', 'concave points_se',
       'texture_worst', 'perimeter_se_kat'], dtype=object)

In [12]:
# Statystyki zmiennych x
train_x.describe()

,perimeter_worst,compactness_worst,concave points_se,texture_worst,perimeter_se_kat
count,352.000000,352.000000,352.000000,352.000000,352.000000
mean,99.919972,0.227180,0.010642,25.072642,0.269886
std,26.403821,0.125834,0.004756,5.816514,0.444532
min,50.410000,0.027290,0.000000,12.020000,0.000000
25%,82.910000,0.134600,0.007307,20.522500,0.000000
50%,92.810000,0.197550,0.010385,25.035000,0.000000
75%,112.200000,0.305500,0.013357,28.717500,1.000000
max,186.800000,0.624700,0.024650,41.850000,1.000000


Nie możemy porównywać pomiędzy sobą współczynników, ponieważ zakres zmiennych jest różny. Do porównania należałoby wystandaryzować zmienne

In [14]:
# Interpretacja zmiennej perimeter_worst
var = 'perimeter_worst'
index = list(model_1.feature_names_in_).index(var)
OR = np.round(np.exp(model_1.coef_[0][index]-1),3)
print(f'Wraz ze wzrostem {var} o jednostkę szansa zajścia zdarzenia wzrasta o {OR*100}%')

Wraz ze wzrostem perimeter_worst o jednostkę szansa zajścia zdarzenia wzrasta o 44.3%


In [18]:
# Interpretacja zmiennej kategorycznej
var = 'perimeter_se_kat'
index = list(model_1.feature_names_in_).index(var)
OR = np.round(1-np.exp(model_1.coef_[0][index]),3)
print(f'W grupie o wartości zmiennej {var}=1 czyli w grupie powyżej średniej perimeter_se szansa zajścia zdarzenia jest niższe o {OR*100}% niż w grupie poniżej średniej.')

W grupie o wartości zmiennej perimeter_se_kat=1 czyli w grupie powyżej średniej perimeter_se szansa zajścia zdarzenia jest niższe o 1.9% niż w grupie poniżej średniej.


## Interacje i przekształcenia zmiennych
Model regresji logistycznej wykrywa jedynie zależności liniowej, dlatego warto testować:
1. przekształcenia zmiennych (np. logarytm, potęga, 1/x,...)
2. Interakcje - iloczyn dwóch zmiennych (mogą też być kategoryczne)


In [19]:
# Iloczyn zmiennej kategorycznej z nominalną
train_x['perim_il'] = train_x['perimeter_se_kat'] * train_x['perimeter_worst']
test_x['perim_il'] = test_x['perimeter_se_kat'] * test_x['perimeter_worst']

In [54]:
# Stworzenie objektu modelu
model_1 = LogisticRegression(C=0.4)

In [20]:
# Estymacja 
model_1.fit(train_x,train_y)

LogisticRegression(C=0.4)

In [21]:
#Współczynniki modelu
model_1.coef_

array([[ 0.18740631,  0.67231744,  0.01607141,  0.22475845,  0.05506212,
        -0.00135343]])

In [22]:
# Nazwy zmiennych
model_1.feature_names_in_

array(['perimeter_worst', 'compactness_worst', 'concave points_se',
       'texture_worst', 'perimeter_se_kat', 'perim_il'], dtype=object)

In [24]:
# Interpretacja interakcji
var = 'perim_il'
index = list(model_1.feature_names_in_).index(var)
OR = np.round(1-np.exp(model_1.coef_[0][index]),3)
print(f'W grupie o wartości zmiennej {var}=1 czyli w grupie powyżej średniej wraz ze wzrostem wartości perimeter_worst o jednostke szansa zajścia zdarzenia jest niższa dodatkowo {OR*100}%.')

W grupie o wartości zmiennej perim_il=1 czyli w grupie powyżej średniej wraz ze wzrostem wartości perimeter_worst o jednostke szansa zajścia zdarzenia jest niższa dodatkowo 0.1%.


Interpretacja przekształcenia zmiennej odbyba się w taki sam sposób jak podstawowyc zmiennych, zmienia się tylko definicja zmiennej.